In [ ]:
# Papilon Example: Marketing Simulation & Insights

from papilon import simulate_kde_scenarios, infer_causal_structure, grid_search_optimize
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Simulate dataset
np.random.seed(0)
df = pd.DataFrame({
    'channel': np.random.choice(['email', 'sms', 'paid'], size=200),
    'spend': np.random.exponential(scale=100, size=200),
    'CTR': np.random.beta(2, 5, 200),
})
df['conversions'] = np.random.binomial(1, df['CTR'] * 0.5)

# Analyze causality
df_encoded = df.copy()
df_encoded['channel'] = df_encoded['channel'].astype('category').cat.codes
graph = infer_causal_structure(df_encoded, visualize=True)

# Optimize
mean_ctr = df_encoded.groupby('channel')['CTR'].mean().to_dict()
def objective_fn(config):
    return mean_ctr[config['channel']] * (config['spend'] / 100) * 0.5

results = grid_search_optimize(df_encoded, {
    'spend': [50, 100, 150],
    'channel': list(mean_ctr.keys())
}, objective_fn)

print(results)